# Setting up the DB

This script should be run once in order to create the database.

In [1]:
from sqlalchemy import create_engine, Table
from sqlalchemy import Column, Integer, String, Text, Date, ForeignKey
from sqlalchemy.ext.declarative import declarative_base
from sqlalchemy.orm import relationship, sessionmaker

In [2]:
engine = create_engine('sqlite:///crm.db', echo=False)
Base = declarative_base()

In [3]:
class Customer(Base):
    __tablename__ = 'customers'
    id = Column(Integer, primary_key=True)
    name = Column(String)
    last_name = Column(String)
    email = Column(String)

    def __repr__(self):
        return "<Customer(name='{}', last_name='{}', email='{}')>".format(
        self.name, self.last_name, self.email)

In [4]:
class Message(Base):
    __tablename__ = 'messages'
    id = Column(Integer, primary_key=True)
    name = Column(String)
    text = Column(Text)
    date = Column(Date)
    
    customer_id = Column(Integer, ForeignKey('customers.id'))
    customer = relationship("Customer", back_populates="messages")

    def __repr__(self):
        return "<Message(name='{}', date='{}', customer='{}')>".format(
        self.name, self.date, self.customer)

In [5]:
class List(Base):
    __tablename__ = 'lists'
    id = Column(Integer, primary_key=True)
    name = Column(String)

In [6]:
association_table = Table('list_customer', Base.metadata,
    Column('left_id', Integer, ForeignKey('customers.id')),
    Column('right_id', Integer, ForeignKey('lists.id'))
)

In [7]:
Customer.messages = relationship('Message', order_by=Message.id, back_populates='customer')
Customer.lists = relationship("List",
                    secondary=association_table,
                    backref="customers")

In [8]:
Base.metadata.create_all(engine)

In [9]:
Session = sessionmaker(bind=engine)
session = Session()